# Experiments with Explainable boosting #

In [1]:
debug_local = True #to use local version
number_of_folds = 5

In [2]:
from pathlib import Path
import sys
import inspect

local = (Path("..") / "yspecies").resolve()
if debug_local and local.exists():
  sys.path.insert(0, Path("..").as_posix())
  #sys.path.insert(0, local.as_posix())
  print("extending pathes with local yspecies")
  print(sys.path)
  %load_ext autoreload
  %autoreload 2

extending pathes with local yspecies
['..', '/data/sources/yspecies/notebooks', '/opt/miniconda3/envs/yspecies/lib/python38.zip', '/opt/miniconda3/envs/yspecies/lib/python3.8', '/opt/miniconda3/envs/yspecies/lib/python3.8/lib-dynload', '', '/opt/miniconda3/envs/yspecies/lib/python3.8/site-packages', '/opt/miniconda3/envs/yspecies/lib/python3.8/site-packages/IPython/extensions', '/home/antonkulaga/.ipython']


In [54]:
from typing import *
from yspecies.dataset import *
from yspecies.utils import *
from yspecies.workflow import *
from yspecies.partition import *
import yspecies
import yspecies.utils

In [4]:
from dataclasses import dataclass
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [5]:
from dataclasses import dataclass
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [6]:
#settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
import pprint
pp = pprint.PrettyPrinter(indent=4)

#charts settings
import matplotlib.pyplot as plt
from IPython.display import set_matplotlib_formats
%matplotlib auto
plt.ioff()
set_matplotlib_formats('svg')

Using matplotlib backend: TkAgg


## Loading data ##

In [7]:
from pathlib import Path
locations: Locations = Locations("./") if Path("./data").exists() else Locations("../")

In [35]:
data = ExpressionDataset.from_folder(locations.interim.selected)
data

expressions,genes,species,samples,Genes Metadata,Species Metadata
"(414, 12352)",12352,40,414,"(12352, 2)","(41, 19)"


In [39]:
data_cleaned = data.collect(lambda e: e.dropna(axis=1))
data_cleaned

expressions,genes,species,samples,Genes Metadata,Species Metadata
"(414, 3896)",3896,40,414,"(3896, 2)","(41, 19)"


In [40]:
from sklearn.pipeline import Pipeline

selection = FeatureSelection(
    samples = ["tissue","species"], #samples metadata to include
    species =  [], #species metadata other then Y label to include
    exclude_from_training = ["species"],  #exclude some fields from LightGBM training
    to_predict = "lifespan", #column to predict
    categorical = ["tissue"])


In [41]:
pipe = Pipeline(
    [
        ('extractor', DataExtractor(selection)),
         ("partitioner", DataPartitioner(folds =  5, species_in_validation=2, not_validated_species = ["Homo_sapiens"])), # to partition it according to sorted stratification
    ]
               )

In [42]:
partitions = pipe.fit_transform(data_cleaned)
partitions

<table><tr><th>partitions_X</th><th>partitions_Y</th></tr><tr><td align='left'>[ (88, 3897),(81, 3897),(78, 3897),(75, 3897),(92, 3897) ]</td><td align='left'>[ (88,),(81,),(78,),(75,),(92,) ]</td></tr><tr><th>show(X,10,10)</th><th>show(Y,10,10)</th></tr><tr><td><div>
<style scoped>
    .dataframe tbody tr th:only-of-type {
        vertical-align: middle;
    }

    .dataframe tbody tr th {
        vertical-align: top;
    }

    .dataframe thead th {
        text-align: right;
    }
</style>
<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>ENSG00000073921</th>
      <th>ENSG00000139687</th>
      <th>ENSG00000135506</th>
      <th>ENSG00000165995</th>
      <th>ENSG00000073756</th>
      <th>ENSG00000097007</th>
      <th>ENSG00000140519</th>
      <th>ENSG00000179915</th>
      <th>ENSG00000068912</th>
      <th>ENSG00000103479</th>
    </tr>
    <tr>
      <th>run</th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>SRR5961830</th>
      <td>465.005</td>
      <td>55.082</td>
      <td>86.345</td>
      <td>137.115</td>
      <td>13.902</td>
      <td>14.974</td>
      <td>4.506</td>
      <td>1437.967</td>
      <td>68.670</td>
      <td>132.300</td>
    </tr>
    <tr>
      <th>SRR5885325</th>
      <td>95.720</td>
      <td>7.969</td>
      <td>92.622</td>
      <td>3.058</td>
      <td>4.627</td>
      <td>12.168</td>
      <td>73.229</td>
      <td>0.196</td>
      <td>19.562</td>
      <td>23.834</td>
    </tr>
    <tr>
      <th>SRR306843</th>
      <td>78.732</td>
      <td>1.167</td>
      <td>71.952</td>
      <td>25.618</td>
      <td>4.684</td>
      <td>8.156</td>
      <td>4.476</td>
      <td>249.804</td>
      <td>18.811</td>
      <td>9.824</td>
    </tr>
    <tr>
      <th>SRR306844</th>
      <td>47.023</td>
      <td>13.521</td>
      <td>254.783</td>
      <td>15.522</td>
      <td>0.705</td>
      <td>6.238</td>
      <td>0.761</td>
      <td>231.219</td>
      <td>14.245</td>
      <td>25.245</td>
    </tr>
    <tr>
      <th>SRR306845</th>
      <td>61.764</td>
      <td>15.757</td>
      <td>152.155</td>
      <td>23.108</td>
      <td>1.287</td>
      <td>9.979</td>
      <td>0.522</td>
      <td>112.169</td>
      <td>27.075</td>
      <td>35.969</td>
    </tr>
    <tr>
      <th>SRR306841</th>
      <td>84.034</td>
      <td>12.452</td>
      <td>87.754</td>
      <td>31.474</td>
      <td>1.816</td>
      <td>8.308</td>
      <td>2.715</td>
      <td>216.067</td>
      <td>64.003</td>
      <td>21.585</td>
    </tr>
    <tr>
      <th>SRR306840</th>
      <td>99.109</td>
      <td>14.010</td>
      <td>84.165</td>
      <td>37.553</td>
      <td>3.502</td>
      <td>19.208</td>
      <td>6.646</td>
      <td>127.598</td>
      <td>46.852</td>
      <td>33.297</td>
    </tr>
    <tr>
      <th>SRR306839</th>
      <td>104.142</td>
      <td>5.173</td>
      <td>92.039</td>
      <td>29.659</td>
      <td>2.401</td>
      <td>17.796</td>
      <td>5.618</td>
      <td>96.630</td>
      <td>31.826</td>
      <td>18.945</td>
    </tr>
    <tr>
      <th>SRR787277</th>
      <td>575.293</td>
      <td>104.400</td>
      <td>123.212</td>
      <td>42.191</td>
      <td>82.374</td>
      <td>50.811</td>
      <td>0.000</td>
      <td>8.258</td>
      <td>56.177</td>
      <td>236.303</td>
    </tr>
    <tr>
      <th>SRR1521445</th>
      <td>78.535</td>
      <td>10.850</td>
      <td>100.439</td>
      <td>22.534</td>
      <td>2.425</td>
      <td>7.839</td>
      <td>2.323</td>
      <td>149.655</td>
      <td>49.243</td>
      <td>20.210</td>
    </tr>
  </tbody>
</table>
</div></td><td><div>
<style scoped>
    .dataframe tbody tr th:only-of-type {
        vertical-align: middle;
    }

    .dataframe tbody tr th {
        vertical-align: top;
    }

    .dataframe thead th {
   

## ExplainableBoostingRegressor ##

In [43]:
from interpret.glassbox import ExplainableBoostingRegressor


In [44]:
X_train, X_test, y_train, y_test = partitions.split_fold(1)

In [58]:
yspecies.utils.show(X_train,10,10)

,ENSG00000073921,ENSG00000139687,ENSG00000135506,ENSG00000165995,ENSG00000073756,ENSG00000097007,ENSG00000140519,ENSG00000179915,ENSG00000068912,ENSG00000103479
run,,,,,,,,,,
SRR5961830,465.005,55.082,86.345,137.115,13.902,14.974,4.506,1437.967,68.670,132.300
SRR306841,84.034,12.452,87.754,31.474,1.816,8.308,2.715,216.067,64.003,21.585
SRR3715877,229.144,49.916,149.272,19.608,17.951,45.862,0.000,2.867,57.933,125.831
SRR5961972,532.382,54.231,100.576,132.921,23.412,20.063,4.173,1319.167,74.790,110.415
SRR306801,71.118,8.574,102.943,32.014,2.416,21.327,4.059,113.476,33.830,20.275
SRR306804,4.728,0.734,23.175,4.504,0.269,2.407,0.000,0.181,3.818,2.157
SRR2040586,79.189,5.313,48.408,3.038,12.198,10.885,0.405,0.628,22.559,3.988
SRR306793,44.370,12.534,39.623,19.499,0.639,7.946,0.123,31.917,10.779,18.883
SRR306834,39.021,2.175,84.626,1.939,1.362,8.243,100.296,0.085,12.490,8.957


In [67]:
X_train.columns.drop("tissues")

Index(['ENSG00000073921', 'ENSG00000139687', 'ENSG00000135506',
       'ENSG00000165995', 'ENSG00000073756', 'ENSG00000097007',
       'ENSG00000140519', 'ENSG00000179915', 'ENSG00000068912',
       'ENSG00000103479',
       ...
       'ENSG00000274523', 'ENSG00000275066', 'ENSG00000275410',
       'ENSG00000275835', 'ENSG00000276023', 'ENSG00000276234',
       'ENSG00000277399', 'ENSG00000278540', 'ENSG00000284862',
       'tissueencoded'],
      dtype='object', length=3897)

In [78]:
fnames = data_cleaned.genes_meta.symbol.values.tolist()
fnames.append("tissueencoded")

In [79]:
seed = 42
ebm = ExplainableBoostingRegressor(random_state=seed, feature_names = fnames)
ebm.fit(X_train, y_train)   #Works on dataframes and numpy arrays

ExplainableBoostingRegressor(feature_names=['PICALM', 'RB1', 'OS9', 'CACNB2',
                                            'PTGS2', 'ABL1', 'RHCG', 'NRXN1',
                                            'ERLEC1', 'RBL2', 'COLEC12',
                                            'SLC45A3', 'COL14A1', 'EML1',
                                            'AFAP1L1', 'RHBG', 'AFAP1', 'RHAG',
                                            'RBL1', 'ALPL', 'FNBP1L', 'DIS3L',
                                            'DTWD1', 'MEOX1', 'ETNK1', 'BCL9L',
                                            'FNBP1', 'MTMR2', 'FAM219A',
                                            'MTMR1', ...],
                             feature_types=['continuous', 'continuous',
                                            'continuous', 'continuous',
                                            'continuous', 'continuous',
                                            'continuous', 'continuous',
                                      

In [80]:
from interpret.perf import *
reg = RegressionPerf(ebm.predict, feature_names = data_cleaned.genes_meta.symbol)
exp = reg.explain_perf(X_test, y_test)
exp.data()

{'type': 'perf_curve',
 'density': {'names': array([-18.01840483,  -9.52790964,  -1.03741446,   7.45308073,
          15.94357591,  24.4340711 ,  32.92456628,  41.41506147,
          49.90555665,  58.39605184,  66.88654702,  75.37704221,
          83.86753739]),
  'scores': array([ 9, 47, 11,  9,  0,  1,  0,  1,  1,  0,  0,  2])},
 'scores': array([40.73369691, 65.7319198 , 38.63246261, 75.35820248, 50.60150207,
        46.74082683, 32.5974618 , 44.54575748, 43.94212326, 46.11911103,
        30.8630359 , 23.65369528, 27.18544085, 35.28101385, 28.47316466,
        27.11670616, 20.39817159, 31.12261143, 23.86702205, 25.57226117,
        22.31833852, 22.55529334, 22.05600379, 23.8939015 , 34.90923554,
        30.55910834, 26.40026731, 22.64182327, 20.08267732, 23.99711141,
        21.87311952, 28.44453305, 23.69242754, 19.888197  , 25.88910075,
        28.76671439, 24.06773351, 23.62912872, 24.1448765 , 23.26828537,
        28.68926877, 24.75675188, 15.27216261, 27.01840483, 10.29069202,


In [84]:
exp.data()["r2"]

0.6102847365655819

In [81]:
from interpret import show

ebm_global = ebm.explain_global()
show(ebm_global)

<!-- http://127.0.0.1:7801/140346888309488/ -->

In [82]:
ebm_local = ebm.explain_local(X_test, y_test)
show(ebm_local)

<!-- http://127.0.0.1:7801/140346479582464/ -->